# CAFEf6_reduce_write_zarr

**Date:** <br>
10 June 2022 <br>
**Background:** <br>
Issue - https://github.com/Thomas-Moore-Creative/NCI-CAFE-ARD/issues/2 <br>
**Author(s):**<br>
Thomas Moore<br>

## We are using NCI OOD as platform for data processing
### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,processes=1,memory="47GB",walltime='03:00:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.148:42505,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Load in CAFEf6 zarr collection

In [36]:
path_in = '/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20151101/'
path_out = '/g/data/xv83/users/tm4888/data/CAFE/hindcast_f6/'

In [26]:
ds_f6 = xr.open_zarr(path_in+'ocean_month.zarr.zip',consolidated=True)

In [27]:
ds_f6.nbytes/1e9

4986.59589072

# 5 TB!

In [28]:
ds_f6

<xarray.Dataset>
Dimensions:                 (time: 120, ensemble: 96, st_ocean: 50, yt_ocean: 300, xt_ocean: 360, yu_ocean: 300, xu_ocean: 360, nv: 2, st_edges_ocean: 51, sw_edges_ocean: 51, sw_ocean: 50)
Coordinates: (12/21)
    area_t                  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    area_u                  (yu_ocean, xu_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    average_DT              (time) timedelta64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T1              (time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T2              (time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
  * ensemble                (ensemble) int64 1 2 3 4 5 6 7 ... 91 92 93 94 95 96
    ...                      ...
  * time                    (time) object 2015-11-16 00:00:00 ... 2025-10-16 ...
    time_bounds             (time, nv) timedelta64[ns] dask.array<chunksize=(120, 2), meta=np.ndarray>
  * xt_ocean                (xt_ocean) float64 -279.5 -278.5 ... 78.5 79.5
  * xu_ocean                (xu_ocean) float64 -279.0 -278.0 ... 79.0 80.0
  * yt_ocean                (yt_ocean) float64 -77.88 -77.63 ... 89.32 89.77
  * yu_ocean                (yu_ocean) float64 -77.75 -77.51 ... 89.55 90.0
Data variables: (12/22)
    age_global              (time, ensemble, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    cfc_11                  (time, ensemble, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    cfc_12                  (time, ensemble, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    eta_t                   (time, ensemble, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 300, 360), meta=np.ndarray>
    neutral_diffusion_salt  (time, ensemble, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    neutral_diffusion_temp  (time, ensemble, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    ...                      ...
    tx_trans_gm             (time, ensemble, st_ocean, yt_ocean, xu_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    ty_trans                (time, ensemble, st_ocean, yu_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    ty_trans_gm             (time, ensemble, st_ocean, yu_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    u                       (time, ensemble, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    v                       (time, ensemble, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
    wt                      (time, ensemble, sw_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 96, 1, 300, 360), meta=np.ndarray>
Attributes:
    filename:   ocean_month.zarr
    grid_tile:  1
    grid_type:  mosaic
    title:      AccessOcean-AM2

# drop variables not in use

In [29]:
ds_f6 = ds_f6.drop({'age_global','cfc_11','cfc_12','eta_t','neutral_diffusion_salt',
                            'neutral_diffusion_temp','neutral_gm_salt','neutral_gm_temp','salt_sponge_tend',
                            'salt_vdiff_impl','temp_sponge_tend','temp_vdiff_impl','tx_trans_gm','ty_trans_gm','tx_trans','ty_trans',
                            'wt','sw_ocean'})

In [30]:
ds_f6.nbytes/1e9

1000.30725032

# 1 TB!

# Reduce to ensemble mean!

In [31]:
ds_f6 = ds_f6.mean(dim = 'ensemble', keep_attrs=True, skipna=True)

In [32]:
ds_f6.nbytes/1e9

10.422449552

# 10 GB $phew!?$


In [33]:
ds_f6

<xarray.Dataset>
Dimensions:         (yt_ocean: 300, xt_ocean: 360, yu_ocean: 300, xu_ocean: 360, time: 120, nv: 2, st_ocean: 50, st_edges_ocean: 51, sw_edges_ocean: 51)
Coordinates: (12/19)
    area_t          (yt_ocean, xt_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    area_u          (yu_ocean, xu_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    geolat_c        (yu_ocean, xu_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    ...              ...
  * time            (time) object 2015-11-16 00:00:00 ... 2025-10-16 12:00:00
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(120, 2), meta=np.ndarray>
  * xt_ocean        (xt_ocean) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * xu_ocean        (xu_ocean) float64 -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * yt_ocean        (yt_ocean) float64 -77.88 -77.63 -77.38 ... 89.32 89.77
  * yu_ocean        (yu_ocean) float64 -77.75 -77.51 -77.26 ... 89.1 89.55 90.0
Data variables:
    salt            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 1, 300, 360), meta=np.ndarray>
    sst             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 300, 360), meta=np.ndarray>
    temp            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 1, 300, 360), meta=np.ndarray>
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(3, 1, 300, 360), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(3, 1, 300, 360), meta=np.ndarray>
Attributes:
    filename:   ocean_month.zarr
    grid_tile:  1
    grid_type:  mosaic
    title:      AccessOcean-AM2

# rechunk to reduce small chunks

In [34]:
ds_f6 = ds_f6.chunk({'time':120})

# crop lead time to 3 years

In [40]:
ds_f6 = ds_f6.isel({'time':slice(0,37)})
ds_f6.nbytes/1e9

3.215389568

# 3.2 GB $phew!?$


# Can we write reduced zarr file at this size?

In [41]:
%%time
ds_f6.to_zarr(path_out+'CAFEf6_reduced_ocean_month.zarr',consolidated=True)

CPU times: user 3min 27s, sys: 14.8 s, total: 3min 42s
Wall time: 10min 23s


# Read back in and test

In [44]:
ds_test = xr.open_zarr(path_out+'CAFEf6_reduced_ocean_month.zarr',consolidated=True)

In [45]:
ds_test

<xarray.Dataset>
Dimensions:         (yt_ocean: 300, xt_ocean: 360, yu_ocean: 300, xu_ocean: 360, time: 37, nv: 2, st_ocean: 50, st_edges_ocean: 51, sw_edges_ocean: 51)
Coordinates: (12/19)
    area_t          (yt_ocean, xt_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    area_u          (yu_ocean, xu_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(37,), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(37,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(37,), meta=np.ndarray>
    geolat_c        (yu_ocean, xu_ocean) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    ...              ...
  * time            (time) object 2015-11-16 00:00:00 ... 2018-11-16 00:00:00
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(37, 2), meta=np.ndarray>
  * xt_ocean        (xt_ocean) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * xu_ocean        (xu_ocean) float64 -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * yt_ocean        (yt_ocean) float64 -77.88 -77.63 -77.38 ... 89.32 89.77
  * yu_ocean        (yu_ocean) float64 -77.75 -77.51 -77.26 ... 89.1 89.55 90.0
Data variables:
    salt            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(37, 1, 300, 360), meta=np.ndarray>
    sst             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(37, 300, 360), meta=np.ndarray>
    temp            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(37, 1, 300, 360), meta=np.ndarray>
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(37, 1, 300, 360), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(37, 1, 300, 360), meta=np.ndarray>
Attributes:
    filename:   ocean_month.zarr
    grid_tile:  1
    grid_type:  mosaic
    title:      AccessOcean-AM2

# $\Downarrow$ Transfered these steps into CAFE60_reduce_write_zarr $\Downarrow$

# Load in reduced CAFE60 zarr collection

In [46]:
ds_CAFEf6 = xr.open_zarr(path_out+'CAFEf6_reduced_ocean_month.zarr',consolidated=True)

In [47]:
ds_CAFEf6.nbytes/1e9

3.215389568

### "T-grid" SST plus temp & salt // 3D diagnostic variables needed to calculate D20, MLD, OHC, T# 

In [48]:
ds_CAFEf6_Tgrid = ds_CAFEf6[['temp','salt','sst']]

### "U-grid" U & V // 3D diagnostic variables needed to calculate EKE#, U100, V100, U100_300, V100_300

In [49]:
ds_CAFEf6_Ugrid = ds_CAFEf6[['u','v']]

# write out U & Tgrid zarr files

In [50]:
%%time
ds_CAFEf6_Tgrid.to_zarr(path_out+'CAFEf6_Tgrid_reduced_ocean_month.zarr',consolidated=True)

CPU times: user 1.41 s, sys: 162 ms, total: 1.58 s
Wall time: 7.56 s


In [51]:
%%time
ds_CAFEf6_Ugrid.to_zarr(path_out+'CAFEf6_Ugrid_reduced_ocean_month.zarr',consolidated=True)

CPU times: user 1.14 s, sys: 196 ms, total: 1.33 s
Wall time: 2.94 s


# $\Uparrow$ Transfered these steps into CAFE60_reduce_write_zarr $\Uparrow$

# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [33]:
client.restart()

<Client: 'tcp://10.0.128.155:40685' processes=8 threads=16, memory=350.16 GiB>

In [52]:
client.shutdown()